In [ ]:
# visualizing specific queries

%load_ext autoreload
%autoreload 2
%matplotlib inline
import torch 
import numpy as np
from PIL import Image
from torchvision import transforms
import pandas as pd 
import json 
import copy 
import pickle 
import os 
import sys
import copy
import umap
import wandb
from pytorch_lightning.loggers import WandbLogger

import matplotlib.pyplot as plt
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
sys.path.append('../py_scripts')
from py_scripts import LightningDataModule, get_params_net_dataloader
import glob
import pickle
import torch.nn.functional as F
from diffusion_utils import *

# DONT NEED TO USE GPU HERE

use_gpu = False

if use_gpu: 
    os.environ["CUDA_VISIBLE_DEVICES"] = "1"

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
else: 
    device="cpu"



In [12]:
bs = 10

nper_row = 25
max_images = nper_row**2

show_receptive_fields = False
save_value_fields = False

min_max_scale = False 
peak_scale = True

save_name_base = "peak_scale_625"

li_model_templates = {
    "19000epochs":'Another6KEpochs_CIFAR10_Adam_lr0.0001_datas=None_10000Neurons_projM=False_nlayers1',
    "9000epochs":'ReconCIFAR10Long_EvenMore_Adam_lr0.0001_datas=None_10000Neurons_projM=False_nlayers1',
    "3000epochs":"ReconCIFAR10Long_Adam_lr0.0001_datas=None_10000Neurons_projM=False_nlayers1"
}
# need to iterate through these. 

#LowerLR
model_template = "Another6KEpochs_CIFAR10_Adam_lr0.0001_datas=None_10000Neurons_projM=False_nlayers1"
#"ReconCIFAR10Long_EvenMore_Adam_lr0.0001_datas=None_10000Neurons_projM=False_nlayers1"
#"InhibCircuitSimple_CIFAR10_Adam_lr0.0001_datas=None_10000Neurons_projM=False_nlayers1"
#"Sigmoid_CIFAR10_Adam_lr0.0001_datas=None_10000Neurons_projM=False_nlayers1"
#"GELU_CIFAR10_Adam_lr0.0001_datas=None_10000Neurons_projM=False_nlayers1"
#"Interpretable_NoCosine_Adam_lr0.0001_datas=None_10000Neurons_projM=False_nlayers1"#Interpretable_Adam_lr0.0001_datas=None_10000Neurons_projM=False_nlayers1"
#"_ffnCIFAR10_w_Projections_Adam_lr0.0001_datas=None_10000Neurons_projM=True_nlayers1"
#"_ffnRaw_CIFAR10_Adam_lr0.0001_datas=None_10000Neurons_projM=False_nlayers1" #"_ffnRaw_CIFAR10_Adam_lr1e-05_datas=None_10000Neurons_projM=False_nlayers1"#"_ffnBaseline_Adam_lr0.0001_datas=None_1steps_10000Neurons_projM=False_nlayers1"

use_CIFAR10 = True

model_prefixes = [0.0,0.05,0.1,0.3,0.8,1.5, 3.0, 10.0] #['0.0001', '0.00001', '0.000001', '0.0000001', 0.0]
#[0.0,0.05,0.1,0.3,0.8,1.5, 3.0, 10.0]
#["100N","1_000N","10_000N","100_000N"]

dataset_path="../data/"
save_dir = "../../scratch_link/Foundational-SDM/data/CachedLatents/"
extra = {"use_wandb":False,"classification":False, "non_relu_act_threshold":0.0001}

num_queries = 3
only_all_receptives = True

if "CIFAR10" in model_template or use_CIFAR10: 
    latent, labels = torch.load('../data/CIFAR10/all_data_train.pt')
    latent = latent.flatten(start_dim=1 )
    if latent.dtype is torch.uint8:#"/ImageNet32/" in self.dataset_path or "/CIFAR10/" in self.dataset_path:
            latent = latent.type(torch.float)/255
else: 

    latent, labels = torch.load('../data/CachedOutputs/ConvMixerWTransforms_ImgNet32_CIFAR10/all_data_train.pt')

q_rand_inds = np.random.choice(len(latent), num_queries, replace=False)

for template_details, model_template in li_model_templates.items():

    fig_save_name = f"{template_details}_{save_name_base}"

    for run_ind, run in enumerate(model_prefixes): 
        print("Noise amount:", run)

        model, params = load_model(f"{run}{model_template}", dataset_path, save_dir, device, extra_extras=extra)

        with torch.no_grad():

            q = model.run_model_till_noise( latent[q_rand_inds].to(device) )
            q_act_neurons = model.run_model_till_neuron_activations(q, apply_noise=False)
            x_recons = model.run_after_noise(q)
            q, q_act_neurons = q.cpu() , q_act_neurons.cpu() 

        xas = model.X_a().weight.cpu().detach()
        xvs = model.X_v().cpu().detach()
        #if not only_all_receptives:

        if run_ind==0 or not only_all_receptives:
            gridshow( latent[q_rand_inds].view(num_queries, 3,32,32), title=f"Original images", save_name=f"query_activated_recpt_fields/{fig_save_name}_Queries" )

        if not only_all_receptives:
        
            gridshow( q.view(num_queries, 3,32,32), title=f"Noisy images" )
            gridshow( x_recons.view(num_queries, 3,32,32), title=f"Reconstruction | Noise amount={run}" )

        num_active = (q_act_neurons>0.0).sum(1)
        print( "====== number of neurons active for each query", num_active)

        act_vals, act_inds = torch.sort(q_act_neurons, dim=1, descending=True)


        dists = cosine_sim_matrices(model.X_a().weight.detach(), q).T
        dist_vals, dist_inds = torch.topk(dists, 5, dim=1)#.max(dim=0) # what neuron is closest to each query. 
        print("dist inds", dist_inds.shape )

        for qind in range(num_queries):

            if not only_all_receptives:
                gridshow( latent[q_rand_inds[qind]].view(1, 3,32,32), title=f"Target Pattern" )
                gridshow( q[qind].view(1, 3,32,32), title=f"Actual Query" )
                gridshow( xas[dist_inds[qind]], title=f"Top 5 Neurons with closest cosine similarity to the query", use_mm_scale=min_max_scale, use_peak_scale=peak_scale )

            #closest cosine sim

            print('activities', act_vals[qind][:num_active[qind]][:max_images], )

            if len(xas[act_inds[qind][:num_active[qind]]])>max_images:
                print(f"MORE THAN {max_images} max_images being used.")
            gridshow( xas[act_inds[qind][:num_active[qind]]][:max_images], title=f"Keys | Sorted by activation value | Noise amount={run}", use_mm_scale=min_max_scale, use_peak_scale=peak_scale, nrow=nper_row, save_name=f"query_activated_recpt_fields/{fig_save_name}_keys__{run}_{qind}", show_plot= show_receptive_fields)
            if not params.classification and save_value_fields:
                gridshow( xvs[act_inds[qind][:num_active[qind]]][:max_images], title=f"Values | Sorted by activation value | Noise amount={run}", use_mm_scale=min_max_scale, use_peak_scale=peak_scale, nrow=nper_row, save_name=f"query_activated_recpt_fields/{fig_save_name}_values_{run}_{qind}", show_plot=show_receptive_fields )

            if not only_all_receptives:

                # relationship between overall neuron activity and what is active for random queries. 
                neuron_active_summer = get_active_neurons(model, latent, device, params.nneurons[0])

                plt.scatter(neuron_active_summer, q_act_neurons[qind], label="all neurons", alpha=0.2)
                plt.scatter(neuron_active_summer[ act_inds[qind][:num_active[qind]] ], act_vals[qind][:num_active[qind]], label="active neurons", alpha=0.2)
                plt.xlabel("Overall Activity")
                plt.ylabel("Activity for this neuron")
                plt.legend()
                plt.title("Active Neurons for this Query as a function of their overall activity")
                plt.show()



In [ ]:
q.shape , q_act_neurons.shape 

In [ ]:
plt.hist( (q_act_neurons>0).sum(1) )

In [ ]:
plt.hist(q_act_neurons[0], bins=50 )
plt.show() 

In [ ]:
q_ind = 0
k=10

vals, inds = torch.topk(q_act_neurons[q_ind], k)


gridshow(latent[q_rand_inds][q_ind].view(3,32,32))
gridshow(q[q_ind].view(1, 3,32,32))
gridshow(x_recons[q_ind].view(1, 3,32,32))
plt.show()
gridshow( min_max_scale(xas[inds]).view(k,3,32,32 ))

gridshow( min_max_scale(xvs[inds]).view(k,3,32,32 ))